<a href="https://colab.research.google.com/github/juanignaciorey/Autocontent_SEO_with_AUTOML/blob/main/AW_01_KeywordResearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 keywords... piense en 5 a 6 cifras sin perder la calidad


 Tenemos que buscar la mayor cantidad de fuentes... sin perder el objetivo

 

### 1. Requeriment

In [ ]:
### Requerimientos
!pip install pandas
!pip install splinter
!conda install splinter

In [ ]:
#@title Instalacion de paquetes
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
#@title Configuracion de Selenium Browser
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
browser.get("https://www.webite-url.com")
## https://stackoverflow.com/questions/56829470/selenium-google-colab-error-chromedriver-executable-needs-to-be-in-path

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: use options instead of chrome_options
  if __name__ == '__main__':


In [ ]:
width =   400 #@param {type:"integer"}
height =   768 #@param {type:"integer"}
# Rezising
browser.set_window_size(width, height)
browser.maximize_window()

In [ ]:
#@title Instalacion de librerias
import os
from selenium.webdriver.common.keys import Keys
import time

warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
#@title Drive Mount
# conecta tu tiempo de ejecución de Google Colab a tu Google Drive
from google.colab import drive
drive.mount('/content/drive')

### 2. Pilar Keywords

Separa las palabras clave con ','

In [ ]:
kwywords = 'com' #@param {type:"string"}

### 3. Serp

In [ ]:
try:
    from googlesearch import search
except ImportError: 
    print("No module named 'google' found")

In [ ]:
# ref: https://www.geeksforgeeks.org/performing-google-search-using-python-code/
# search(query, tld='com', lang='en', num=10, start=0, stop=None, pause=2.0)

# tld stands for top level domain which means we want to search our result on google.com or google.in or some other domain.
tld = 'com' #@param {type:"string"}
# lang : lang stands for language. 
# https://www.labnol.org/code/19899-google-translate-languages
lang = 'es' #@param ['es', 'en']
# Number of results we want.
num = 10 #@param {type:"integer"}
# start : First result to retrieve.
start = 0 
# stop : Last result to retrieve. Use None to keep searching forever.
stop = None
# pause : Lapse to wait between HTTP requests. Lapse too short may cause Google to block your IP. Keeping significant lapse will make your program slow but its safe and better option.
pause = 2.0

In [ ]:
# query : query string that we want to search for.
query = "download book * filetype:epub"
# Return : Generator (iterator) that yields found URLs. If the stop parameter is None the iterator will loop forever.
for j in search(query, tld, lang, num, stop, pause):
    print(j)

In [ ]:
#@title
import os
import re
import sys
import json
import time
import datetime
import platform
from docopt import docopt
from tqdm import tqdm 
from time import sleep
import pandas as pd
from pandas.io.json import json_normalize
import logging
from jinja2 import Environment, FileSystemLoader

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException

''' 
Visualizza una barra di caricamento per mostrare l'attesa
'''
def sleepBar(seconds):
    for i in tqdm(range(seconds)):
        sleep(1)

def prettyOutputName(filetype='html'):
    _query = re.sub('\s|\"|\/|\:|\.','_', query.rstrip())
    prettyname = _query
    ts = time.time()
    st = datetime.datetime.fromtimestamp(ts).strftime('%d-%m-%Y_%H-%M-%S-%f')
    if filetype != 'html':
        prettyname += "_" + st + "." + filetype
    else:
        prettyname += "_" + st + "." + filetype
    return prettyname


def initBrowser(headless=False):
    if "Windows" in platform.system():
        chrome_path = "driver/chromedriver.exe"
    else:
        chrome_path = "driver/chromedriver"
    chrome_options = Options()
    chrome_options.add_argument("--disable-features=NetworkService")
    if headless:
        chrome_options.add_argument('headless')
    return webdriver.Chrome(options=chrome_options,executable_path=chrome_path)
"""
Search on Google and returns the list of PAA questions in SERP.
"""
def newSearch(browser,query):
    if lang== "en":
        browser.get("https://www.google.com?hl=en")
        searchbox = browser.find_element_by_xpath("//input[@aria-label='Search']")
    else:
        browser.get("https://www.google.com?hl=es")
        searchbox = browser.find_element_by_xpath("//input[@aria-label='Buscar']")
    
    searchbox.send_keys(query)
    sleepBar(2)
    tabNTimes()
    if lang== "en":
        searchbtn = browser.find_elements_by_xpath("//input[@aria-label='Google Search']")
    else:
    	searchbtn = browser.find_elements_by_xpath("//input[@aria-label='Buscar con Google']")
    try:
        searchbtn[-1].click()
    except:
        searchbtn[0].click()
    sleepBar(2)
    paa = browser.find_elements_by_xpath("//span/following-sibling::div[contains(@class,'match-mod-horizontal-padding')]")
    hideGBar()
    return paa
"""
Helper function that scroll into view the PAA questions element.
"""
def scrollToFeedback():
    if lang == "en":
        el = browser.find_element_by_xpath("//div[@class='kno-ftr']//div/following-sibling::a[text()='Feedback']")
    else:
    	el = browser.find_element_by_xpath("//div[@class='kno-ftr']//div/following-sibling::a[text()='Enviar comentarios']")

    actions = ActionChains(browser)
    actions.move_to_element(el).perform()
    browser.execute_script("arguments[0].scrollIntoView();", el)
    actions.send_keys(Keys.PAGE_UP).perform()
    sleepBar(1)
"""
Accessibility helper: press TAB N times (default 2)
"""
def tabNTimes(N=2):
    actions = ActionChains(browser) 
    for _ in range(N):
        actions = actions.send_keys(Keys.TAB)
    actions.perform()

"""
Click on questions N times
"""
def clickNTimes(el, n=1):
    for i in range(n):
        el.click()
        logging.info(f"clicking on ... {el.text}")
        sleepBar(1)
        scrollToFeedback()
        try:
            el.find_element_by_xpath("//*[@aria-expanded='true']").click()
        except:
            pass
        sleepBar(1)

"""
Hide Google Bar to prevent ClickInterceptionError
"""
def hideGBar():
	try:
		browser.execute_script('document.getElementById("searchform").style.display = "none";')
	except:
		pass

"""
Where the magic happens
"""
def crawlQuestions(start_paa, paa_list, initialSet, depth=0):
    _tmp = createNode(paa_lst=paa_list, name=query, children=True)
    
    outer_cnt = 0
    for q in start_paa:
        scrollToFeedback()
        if "Dictionary" in q.text:
            continue
        test = createNode(paa_lst=paa_list, n=0,
                        name=q.text,
                        parent=paa_list[0]["name"],
                        children=True)
        
        clickNTimes(q)
        new_q = showNewQuestions(initialSet)
        for l, value in new_q.items():
            sleepBar(1)
            logging.info(f"{l}, {value.text}")
            test1 = createNode(paa_lst=test[0]["children"][outer_cnt]["children"], 
                                name=value.text,
                                parent=test[0]["children"][outer_cnt]["name"],
                                children=True)
            
        initialSet = getCurrentSERP()
        logging.info(f"Current count: {outer_cnt}")
        outer_cnt += 1
        if depth==1:
            for i in range(depth):
                currentQuestions = []
                for i in initialSet.values():
                    currentQuestions.append(i.text)
                for i in paa_list[0]["children"]:
                    for j in i["children"]:
                        parent = j['name']
                        logging.info(parent)
                        _tmpset = set()
                        if parent in currentQuestions:
                            try:
                                if "'" in parent:
                                    xpath_compiler = '//div[text()="' + parent + '"]'
                                else: 
                                    xpath_compiler= "//div[text()='" + parent + "']"
                                question= browser.find_element_by_xpath(xpath_compiler)
                            except NoSuchElementException:
                                continue
                            scrollToFeedback()
                            sleepBar(1)
                            clickNTimes(question)
                            new_q = showNewQuestions(initialSet)
                            for l, value in new_q.items():
                                if value.text == parent:
                                    continue
                                j['children'].append({"name": value.text,"parent": parent})
                                
                            initialSet = getCurrentSERP()

"""
Get the current Result Page.
Returns: 
    A list with newest questions.
"""
def getCurrentSERP():
    _tmpset = {}
    new_paa = browser.find_elements_by_xpath("//span/following-sibling::div[contains(@class,'match-mod-horizontal-padding')]")
    cnt= 0
    for q in new_paa:
        _tmpset.update({cnt:q})
        cnt +=1
    newInitialSet = _tmpset
    return newInitialSet

"""
Shows new questions.
Args:
    intialSet (dict): The initial set in the PAA box.
Returns:
    list of questions with first 3-4 questions deleted (initalSet).
"""
def showNewQuestions(initialSet):
    tmp = getCurrentSERP()
    deletelist = [k for k, v in initialSet.items() if k in tmp and tmp[k] == v]
    _tst = dict.copy(tmp)
    for i,value in tmp.items():
        if i in deletelist:
            _tst.pop(i)
    return _tst

"""
Create a new node in the list.
Args:
    paa_list_elements: list of web elements
    n: index of 'children' list on a main node
    name: node nome
    parent: Indicates if the node has a parent. Default to null only for first level.
    chilren: Indicates if the node has a children list. default false
Returns:
    list of questions with the current new node
"""
def createNode( n=-1, parent='null', children=False, name='',*, paa_lst):
    logging.info(paa_lst)
    if children:
        _d = {
        "name": name,
        "parent": parent,
        "children": [] 
        }
    else:
        _d = {
        "name": name,
        "parent": parent
        }
    if n!=-1:
        logging.info(paa_lst[n]["children"])
        paa_lst[n]["children"].append(_d)
    else:
        paa_lst.append(_d)
    

    return paa_lst

"""
This func takes in input JSON data and returns csv file.
"""
def flatten_csv(data,depth,prettyname):
    try:
        if depth == 0:
            _ = json_normalize(data[0]["children"], 'children', ['name', 'parent',['children',]], record_prefix='inner.')
            _.drop(columns=['children','inner.children','inner.parent'], inplace=True)
            columnTitle = ['parent','name','inner.name']
            _ = _.reindex(columns=columnTitle)
            _.to_csv(prettyname,sep=";",encoding='utf-8')
        elif depth == 1:
            df = json_normalize(data[0]["children"], meta=['name','children','parent'], record_path="children", record_prefix='inner.')
            frames = [ json_normalize(i) for i in df['inner.children'] ]
            result = pd.concat(frames)
            result.rename(columns={"name": "inner.inner.name", "parent": "inner.name"}, inplace=True)
            merge = pd.merge(df, result, how='left', on="inner.name")
            merge.drop(columns=['name'], inplace=True)
            columnTitle = ['parent','inner.parent','inner.name','inner.inner.name']
            merge = merge.reindex(columns=columnTitle)
            merge = merge.drop_duplicates(subset='inner.inner.name', keep='first')
            merge.to_csv(prettyname,sep=';',encoding='utf-8')
    except Exception as e:
        logging.warning(f"{e}")

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)
    args = docopt(usage)
    print(args)
    MAX_DEPTH = 1

    if args['<depth>']:
        depth = int(args['<depth>'])
        if depth > MAX_DEPTH:
            sys.exit("depth not allowed")
    else:
        depth = 0

    if args['en']:
        lang = "en"
    elif args['es']:
        lang = "es"
        


    if args['<keyword>']:
        if args['--headless']:
            browser = initBrowser(True)
        else:
            browser = initBrowser()
        query = args['<keyword>']
        start_paa = newSearch(browser,query)

        initialSet = {}
        cnt= 0
        for q in start_paa:
            initialSet.update({cnt:q})
            cnt +=1

        paa_list = []

        crawlQuestions(start_paa, paa_list, initialSet,depth)
        treeData = 'var treeData = ' + json.dumps(paa_list) + ';'
        
        if paa_list[0]['children']:
            root = os.path.dirname(os.path.abspath(__file__))
            templates_dir = os.path.join(root, 'templates')
            env = Environment( loader = FileSystemLoader(templates_dir) )
            template = env.get_template('index.html')
            filename = os.path.join(root, 'html', prettyOutputName())
            with open(filename, 'w') as fh:
                fh.write(template.render(
                    treeData = treeData,
                ))

    if args['--csv']:
        if paa_list[0]['children']:
            _path = 'csv/'+prettyOutputName('csv')
            flatten_csv(paa_list, depth, _path)

    browser.close()

NameError: ignored

In [ ]:
# python gquestions.py query <keyword> (en|es) [depth <depth>] [--csv] [--headless]

In [ ]:
form = Slider()
form._repr_html_()

'\n    <input type="range" id="in140498676663184"\n     min="0" max="100" step="1"\n     value="0" \n     oninput="out140498676663184.value=in140498676663184.value"\n     onchange="\n       google.colab.kernel.invokeFunction(\n         \'set_var\', [140498676663184, this.valueAsNumber],\n         {})\n     "\n    >\n    <output id="out140498676663184">0</output>\n    '

### 4. Trends

In [ ]:
from pytrends.request import TrendReq
import pandas as pd
import time
startTime = time.time()
pytrend = TrendReq(hl='en-GB', tz=360)

dataset = []
keywords = query_list

for x in range(0,len(df2)):
     keywords = [df2[x]]
     pytrend.build_payload(
     kw_list=keywords,
     cat=0,
     timeframe='2020-04-01 2020-05-01',
     geo='GB')
     data = pytrend.interest_over_time()
     if not data.empty:
          data = data.drop(labels=['isPartial'],axis='columns')
          dataset.append(data)

result = pd.concat(dataset, axis=1)
result.to_csv('search_trends.csv')

executionTime = (time.time() - startTime)
print('Execution time in sec.: ' + str(executionTime))

####  Ebooks

##### Dorks
name_of_ebook filetype:pdf
name_of_ebook filetype:epub
name_of_ebook filetype:mobi
name_of_ebook filetype:txt
intitle:index.of (epub)
intitle:"calibre library" inurl:browse

##### Webs
https://www.overdrive.com/
http://libgen.rs/
https://www.gutenberg.org/
https://centslessbooks.com/

https://bookboon.com/es
http://free-ebooks.net/
https://freecomputerbooks.com/
https://manybooks.net/

##### Dispositivos
Amazon kindle
Audible
Google Play books
Kobo books

In [ ]:
nombres_posibles = []
filetypes = ['pdf','epub','mobi','txt']

### 2. Medium Posts

In [ ]:
keyword = "data%20science"
url = "https://medium.com/search?q="+keyword
print(url)

In [ ]:
wd.get(url)

ScrollNumber = 50
for i in range(1,ScrollNumber):
    wd.execute_script("window.scrollTo(1,50000)")
    time.sleep(5)

file = open('DS.html', 'w')
file.write(wd.page_source)
file.close()

wd.close()

In [ ]:
import urllib3
from bs4 import BeautifulSoup
import csv
http=urllib3.PoolManager()

def scrape():
    data = open('DS.html','r')
    soup = BeautifulSoup(data, 'html.parser')
    for links in soup.find_all('div', {'class': 'postArticle-readMore'}):
        link = links.find('a').get('href')
        Blog_post(link)

def Blog_post(link):
    try:
        print(link)
        blogData = http.request('GET', link)
        soup = BeautifulSoup(blogData.data, 'html.parser')
        article = ''
        tags = []
        heading = soup.find('h1').text
        for para in soup.find_all('p'):
            p = para.text
        p = p.strip('/u')
        article = article + ' ' + p
        for mtags in soup.find_all('a', {'class ': 'link u - baseColor — link'}):
            tags.append(mtags.text)
            # CreateDataFrame(list())
            someList = [heading, article, tuple(tags)]
            # print(someList[0])
            CreateDataFrame(someList)
    except:
        pass

In [ ]:
search_bar.fill("serpstat.com")
# Now let's set up code to click the search button!
search_button_xpath = '//*[@id="tsf"]/div[2]/div[3]/center/input[1]'
search_button = browser.find_by_xpath(search_button_xpath)[0]
search_button.click()

In [ ]:
### Google Also Ask

### Google Sugest

### Google Pin badage
#### Analizar las primeras 
#### Analizar bbusquedas relacionadas desde acá
##### https://www.google.com/search?q=descargar+gta+vice+city+espa%C3%B1ol&rlz=1C1CHBF_esAR932AR932&sxsrf=ALeKk01KrKANaiNvjzZaJH25ZiXnRYqEcA:1616707508424&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjj1vrlsMzvAhXuIbkGHRV8A9wQ_AUoAnoECAEQBA&biw=1920&bih=937

### Adwords

### Translate - Para hacer canonical content

### Keywords de top(competencia) del tema

### 

In [ ]:
def is_valid(mediafire_url):
    """
    Check is the link to mediafire is no broken
    Example of a broken link: 'https://www.mediafire.com/file/null/oretresf-01.mp4/file'
    """
    valid = True
    link_parts = mediafire_url.split("/")
    if (link_parts[4] == "null"):
        valid = False

    return valid

is_valid = is_valid("url")
is_valid

False

Deberiamos encontrar keywords desde distintas fuentes

In [ ]:
!pip install tensorflow-gpu==1.13.1

In [ ]:
!pip install ludwig

In [ ]:
!gsutil cp gs://dataset-uploader/bbc/bbc-text.csv

CommandException: Wrong number of arguments for "cp" command.


In [ ]:
input_features:
        name: text
        type: text
        level: word
        encoder: parallel_cnn

output_features:
        name: category
        type: category

In [ ]:
!ludwig experiment \
  --data_csv bbc-text.csv\
  --model_definition_file model_definition.yaml

In [ ]:
#https://github.com/uber/ludwig/issues/267#issuecomment-497304317
from ludwig import visualize
visualize.learning_curves(['/content/results/experiment_run_0/training_statistics.json'],None)